# Copy Datasets Using Globus


Each attendee has access to their own Guest Collection, which is a shared endpoint that can be used to host and share data.


**TODO**

* Explain how to login
* Explain how to find the Guest Collection

In [ ]:
import os
current_folder = %pwd
if current_folder != "cheapandfair-template":
    %cd cheapandfair-template

In [ ]:
import json
import copy_dataset

For this tutorial, we assume you're putting all of the datasets under a single folder. This isn't necessary, it just makes things easier.

Modify the following cell and execute it to create the configuration file `config.toml`:

`UUID` is the UUID of the Guest Collection. You can find this and the `DOMAIN` by navigating to the Guest Collection in the Globus web app and look at the settings page.
The same variables are also needed for the Source Collection, which is the endpoint where the data is currently stored. These do not need to be modified for the purposes of this tutorial.

Define your collection's base folder using the `FOLDER` variable. Reasonable names include "datasets", "data", or "repository". These forlder names are just for hierachical organization.

In [ ]:
%%file config.toml
# The following refer to the destination collection, where the data will be copied to and that will serve as backend for the data portal
UUID='xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
FOLDER='/datasets/'
DOMAIN='xxxxxxxxxxxxxxxxxxxx.data.globus.org'
# collection of the source data, during the tutorial this is the "Cheap and FAIR Tutorial Datasets" collection
SOURCE_UUID='7352d991-b0a0-49a2-830c-e8fe8c968ca2'
SOURCE_FOLDER='/public/datasets/'

## Authenticate with Globus

Login to Globus using the following command. This will open a browser window where you can login to Globus and receive a token to paste back into the notebook.

In [ ]:
copy_dataset.login();

## Copy Datasets and create manifests
We copy the dataset by providing the name of the dataset, the UUID of the destination Guest Collection, and the folder we want it copied to.

The method will return the manifest of the files that were copied and will also write a copy of the manifest to the local directory named `{dataset}-manifest.json`.

In [ ]:
cmb_manifest = copy_dataset.copydataset('cmb')

Let's look at a couple of the entries in the file manifest.

In [ ]:
print(json.dumps(cmb_manifest[:2], indent=2))

Now we can copy the other two datasets. You won't need to login again because the tokens have been cached in `~/.cheapandfair.json`.

In [ ]:
dust_manifest = copy_dataset.copydataset('dust')
synch_manifest = copy_dataset.copydataset('synch')

We can see that the manifests were saved locally.

In [ ]:
!ls *.json

TODO now that we have the datasets copied

* Explain how to configure permissions on each folder
* create a group earlier in the notebook and put the UUID in the config


